## Zuber

##### 1. Initialization

In [1]:
# Import libraries that might be necessary

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import statsmodels.api as sm
from scipy.interpolate import UnivariateSpline
import scipy.stats as stats

# Import DataFrams
comp_count_trip = 'https://raw.githubusercontent.com/DHE42/zuber/refs/heads/main/moved_project_sql_result_01.csv'
comp_count_trip_df = pd.read_csv(comp_count_trip)

dropoff_avg_trip = 'https://raw.githubusercontent.com/DHE42/zuber/refs/heads/main/moved_project_sql_result_04.csv'
dropoff_avg_trip_df = pd.read_csv(dropoff_avg_trip)

Above, I have imported necessary libraries and the two datasets I'll be working with. Below, I will review the data.

##### 2. Data Review

In [2]:
# Review  comp_count_trip_df
print("Head of comp_count_trip_df")
print()
comp_count_trip_df.head()
print()

print("Info of comp_count_trip_df")
print()
print(comp_count_trip_df.info())
print()

print("Describe comp_count_trip_df")
print()
comp_count_trip_df.info()
print()

print("Describe comp_count_trip_df")
print()
comp_count_trip_df.describe()
print()

print("Null values of comp_count_trip_df")
comp_count_trip_df.isnull().sum()



Head of comp_count_trip_df


Info of comp_count_trip_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   company_name  64 non-null     object
 1   trips_amount  64 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ KB
None

Describe comp_count_trip_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   company_name  64 non-null     object
 1   trips_amount  64 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ KB

Describe comp_count_trip_df


Null values of comp_count_trip_df


company_name    0
trips_amount    0
dtype: int64

In [3]:
# Review dropoff_avg_trip_df
dropoff_avg_trip_df.head()
dropoff_avg_trip_df.info()
dropoff_avg_trip_df.describe()
dropoff_avg_trip_df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   dropoff_location_name  94 non-null     object 
 1   average_trips          94 non-null     float64
dtypes: float64(1), object(1)
memory usage: 1.6+ KB


dropoff_location_name    0
average_trips            0
dtype: int64